In [1]:
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore') # somente para ignorar os warnings
from tqdm import tqdm_notebook as tqdm # barra de progresso
import psycopg2

In [2]:
gdf = gpd.read_file("amostras.geojson")

In [3]:
gdf

,LULC,geometry
0,0,"POLYGON ((-49.42289 -20.73536, -49.42285 -20.7..."
1,0,"POLYGON ((-49.64354 -20.74755, -49.64313 -20.7..."
2,0,"POLYGON ((-49.59064 -20.69422, -49.59028 -20.6..."
3,0,"POLYGON ((-49.49064 -20.69732, -49.49064 -20.6..."
4,0,"POLYGON ((-49.41948 -20.72410, -49.41910 -20.7..."
...,...,...
523,1,"POLYGON ((-48.64985 -20.68600, -48.64952 -20.6..."
524,1,"POLYGON ((-48.65862 -20.68487, -48.65849 -20.6..."
525,1,"POLYGON ((-48.65712 -20.68539, -48.65680 -20.6..."
526,1,"POLYGON ((-48.65898 -20.58830, -48.65866 -20.5..."


In [4]:
def conecta_db():
    con = psycopg2.connect(host='0.0.0.0', 
                         database='gis',
                         user='docker',
                         password = 'docker',
                         port = 25432)
    return con

In [5]:
def inserir_db(sql,params):
    con = conecta_db()
    cur = con.cursor()
    #try:
    cur.execute(sql,params)
    con.commit()
    cur.close()

In [6]:
gdf['geometry'] = gdf['geometry'].apply(lambda x: str(x)[9:-1].replace('(','').replace(')',''))

In [7]:
gdf

,LULC,geometry
0,0,"-49.42289299393238 -20.73536312895599, -49.422..."
1,0,"-49.64353995566358 -20.747549817902662, -49.64..."
2,0,"-49.59063982144597 -20.69422238967157, -49.590..."
3,0,"-49.490636297627546 -20.697318687997623, -49.4..."
4,0,"-49.41948456607041 -20.724101399315774, -49.41..."
...,...,...
523,1,"-48.64985413002123 -20.68600305131959, -48.649..."
524,1,"-48.65861958625936 -20.684867217306167, -48.65..."
525,1,"-48.657117549214384 -20.685389152212498, -48.6..."
526,1,"-48.65898469631072 -20.58829677779559, -48.658..."


In [16]:
pbar = tqdm(total = len(gdf))

for i in gdf.index:
    sql = """
    INSERT into public.amostras (idlulcn,coordenadas,geom) 
    values(%s,%s,ST_GeomFromText(%s, 4326));
    """
    params = [int(gdf['LULC'][i]),gdf['geometry'][i],'POLYGON(({}))'.format(gdf['geometry'][i])]
    inserir_db(sql,params)
    
    pbar.update(1)

  0%|          | 0/528 [00:00<?, ?it/s]

In [15]:
len(str(gdf['geometry'][3]))

201